In [3]:
"""h5形式のモデルをSavedModelへ変換し、SageMakerのエンドポイントへデプロイする

ref: https://dev.classmethod.jp/articles/vpc-access-sagemaker-notebook/
"""
import boto3
import json
import os 
 
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow.serving import Model
 
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import model_from_json, load_model
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
 
# IAMロール定義
role = get_execution_role()

In [5]:
model = load_model('model.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


[2022-04-28 06:24:38.639 ip-172-16-41-154:25724 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-04-28 06:24:38.729 ip-172-16-41-154:25724 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [6]:
input_sig  = 'inputs'
output_sig = 'score'
signature = predict_signature_def(inputs={input_sig: model.input}, outputs={output_sig:model.output})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [9]:
model_version = '1'
export_path   = './tf_model/mnist/' + model_version
if not os.path.exists(export_path):
    os.mkdir(export_path)
 
with K.get_session() as sess:
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)
    builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING], signature_def_map={'serving_default': signature}, main_op=tf.tables_initializer())
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./tf_model/mnist/1/saved_model.pb


In [10]:
!saved_model_cli show --dir $export_path --all

/bin/sh: saved_model_cli: command not found


In [31]:
!cd /home/ec2-user/SageMaker
! tar zcvf model.tar.gz  tf_model/mnist tf_model/code

tf_model/mnist/
tf_model/mnist/1/
tf_model/mnist/1/saved_model.pb
tf_model/mnist/1/variables/
tf_model/mnist/1/variables/variables.index
tf_model/mnist/1/variables/variables.data-00000-of-00001
tf_model/code/
tf_model/code/inference.py


In [32]:
sagemaker_session  = sagemaker.Session()
model_data = sagemaker_session.upload_data(path='tf_model/model.tar.gz', key_prefix='model')
print("done")

done


In [34]:
tensorflow_serving_mnist_model = Model(model_data=model_data,
                                        role=role,
                                        framework_version='1.15.5')
predictor = tensorflow_serving_mnist_model.deploy(initial_instance_count=1,
                                                    instance_type='ml.t2.medium',
                                                    endpoint_name='mnist-endpoint')

The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


------!

In [62]:
import pickle as pkl
img = pkl.load(open("data/mnist.pickle", "rb"))
len(img[0][0]), len(img[0][1])
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_train /= 255
client = boto3.client("sagemaker-runtime")
res = client.invoke_endpoint(EndpointName=predictor.endpoint_name,
                            Body=json.dumps({'inputs':[x_train[0].tolist()]}),
                            ContentType='application/json',
                            Accept='application/json')
result = json.loads(res['Body'].read())

In [63]:
result

{'outputs': [[1.31569811e-10,
   4.00890077e-09,
   7.53057269e-11,
   0.0169722475,
   1.24603617e-11,
   0.983027697,
   7.75427367e-09,
   1.49165791e-10,
   4.87038925e-08,
   5.02105415e-08]]}